# Meclis Soruşturma - Data Collection

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import time
import os

In [2]:
driver = webdriver.Chrome()
driver.implicitly_wait(30)

## Meclis Soruşturma

- source_page: https://www.tbmm.gov.tr/Denetim/Gecmis-Donem-Meclis-Sorusturma-Onergeleri
- donem_values:
    - 22. Dönem Tüm Yasama Yılları: **221, 222, 223, 224, 225**
    - 23. Dönem Tüm Yasama Yılları: **231, 232, 234, 235**
    - 24. Dönem Tüm Yasama Yılları: **241, 242, 243, 244, 245**
    - 25. Dönem Tüm Yasama Yılları: **251, 252**
    - 26. Dönem Tüm Yasama Yılları: **261, 262, 263**
- select_donem (Dönemi ve Yasama Yılı) XPATH: //*[@id="DonemYasamaYili"]
- input_search (Sorguyu Başlat) XPATH: //*[@id="main-area"]/form/div[9]/div/input[2]

In [3]:
def get_sonuc_page(source_page, donem):

    # Open source page
    driver.get(source_page)
    
    # Select Dönem
    select_donem = Select(driver.find_element(By.XPATH, """//*[@id="DonemYasamaYili"]"""))
    donem = str(donem)
    select_donem.select_by_value(donem)
    
    # Find "Sorguyu Başlat" button
    input_search = driver.find_element(By.XPATH, """//*[@id="main-area"]/form/div[9]/div/input[2]""")

    # Click "Sorguyu Başlat" button
    input_search.click()
    
    # Get total page size
    pagination = driver.find_element(By.XPATH, """//*[@id="dataTable_paginate"]/ul""")
    pagination_elements = pagination.find_elements(By.TAG_NAME, "li")
    
    # if there are no records:
    if len(pagination_elements) == 2:
        page_size = 0
    # if there is one page
    elif len(pagination_elements) == 3:
        page_size = 1
    # if there are two pages:
    elif len(pagination_elements) == 4:
        page_size = 2
    # if there are more than two pages
    else:
        page_size = int(pagination_elements[-2].text)
    
    return page_size

In [4]:
def collect_urls():
    
    urls = []
    
    # Find table
    table = driver.find_element(By.XPATH, """//*[@id="dataTable"]/tbody""")
    
    # Get rows of table
    rows = None
    while rows is None:
        try:
            rows = table.find_elements(By.TAG_NAME, "tr")
        except:
            pass
        
    for i, row in enumerate(rows):
        url = row.find_elements(By.TAG_NAME, "td")[3].find_elements(By.TAG_NAME, "div")[3].find_element(By.TAG_NAME, "a").get_attribute("href")
        urls.append(url)
        
    return urls

In [5]:
donemler = [221, 222, 223, 224, 225, 231, 232, 234, 235, 241, 242, 243, 244, 245, 251, 252, 261, 262, 263]
source_page = "https://www.tbmm.gov.tr/Denetim/Gecmis-Donem-Meclis-Sorusturma-Onergeleri"

In [6]:
# Create urls folder if not exists
if "urls" not in os.listdir():
    os.mkdir("urls")

# Write urls to txt file
with open("urls/meclis_sorusturma_urls.txt", "a+") as f:
    for donem in donemler:
        # Get sonuc page & page size
        page_size = get_sonuc_page(source_page, donem)
        print(f"Dönem: {donem} | Page Size: {page_size} ... ", end="")
    
        page = 1
        
        while page <= page_size:
            try:
                time.sleep(1)
                # Collect urls in the page
                for url in collect_urls():
                    f.write(f"{url}\n")
                
                # Find "Sonraki" button
                next_button = driver.find_element(By.XPATH, """//*[@id="dataTable_next"]""")
                
                # Click "Sonraki" button
                next_button.click()
                
                page += 1
                
            except:
                time.sleep(5)
                continue
        print("Completed!")

Dönem: 221 | Page Size: 1 ... Completed!
Dönem: 222 | Page Size: 1 ... Completed!
Dönem: 223 | Page Size: 1 ... Completed!
Dönem: 224 | Page Size: 0 ... Completed!
Dönem: 225 | Page Size: 1 ... Completed!
Dönem: 231 | Page Size: 1 ... Completed!
Dönem: 232 | Page Size: 1 ... Completed!
Dönem: 234 | Page Size: 1 ... Completed!
Dönem: 235 | Page Size: 0 ... Completed!
Dönem: 241 | Page Size: 0 ... Completed!
Dönem: 242 | Page Size: 0 ... Completed!
Dönem: 243 | Page Size: 1 ... Completed!
Dönem: 244 | Page Size: 1 ... Completed!
Dönem: 245 | Page Size: 1 ... Completed!
Dönem: 251 | Page Size: 0 ... Completed!
Dönem: 252 | Page Size: 0 ... Completed!
Dönem: 261 | Page Size: 0 ... Completed!
Dönem: 262 | Page Size: 0 ... Completed!
Dönem: 263 | Page Size: 0 ... Completed!


In [7]:
# Read urls from txt file
with open("urls/meclis_sorusturma_urls.txt", "r") as f:
    urls = [url.strip() for url in f.readlines()]

In [8]:
def get_tables(url):
    
    driver.get(url)
    
    main_area = driver.find_element(By.XPATH, """//*[@id="main-area"]""")
    
    meclis_sorusturma_bilgileri_df = pd.DataFrame()
    meclis_sorusturma_imza_sahipleri_df = pd.DataFrame()
    
    
    for i in range(len(main_area.find_elements(By.TAG_NAME, "div"))):
        if main_area.find_elements(By.TAG_NAME, "span")[i].text == "Geçmiş Dönem Meclis Soruşturması Önerge Bilgileri":
            meclis_sorusturma_bilgileri = main_area.find_elements(By.TAG_NAME, "div")[i]
            meclis_sorusturma_bilgileri_tr_tags = meclis_sorusturma_bilgileri.find_elements(By.TAG_NAME, "tr")
            meclis_sorusturma_bilgileri_df = pd.DataFrame(pd.Series({meclis_sorusturma_bilgileri_tr_tags[i].find_elements(By.TAG_NAME, "td")[0].text: meclis_sorusturma_bilgileri_tr_tags[i].find_elements(By.TAG_NAME, "td")[1].text for i in range(1, len(meclis_sorusturma_bilgileri_tr_tags))})).T
            meclis_sorusturma_id = ''.join(meclis_sorusturma_bilgileri_df.iloc[0, 0].split(" / ") + meclis_sorusturma_bilgileri_df.iloc[0, 1].split(" / "))
            meclis_sorusturma_bilgileri_df.columns = ["donemi_ve_yasama_yili", "esas_numarasi", "baskanliga_gelis_tarihi", "onergenin_ozeti", "okutuldugu_birlesim", "son_durumu", "onergenin_gorusuldugu_tarih", "onergenin_muhatabi", "onerge_sahibi"]
            meclis_sorusturma_bilgileri_df["meclis_sorusturma_id"] = meclis_sorusturma_id
            
        if main_area.find_elements(By.TAG_NAME, "span")[i].text == "İmza Sahibi Milletvekilleri":
            meclis_sorusturma_imza_sahipleri = main_area.find_elements(By.TAG_NAME, "div")[i]
            meclis_sorusturma_imza_sahipleri_tr_tags = meclis_sorusturma_imza_sahipleri.find_elements(By.TAG_NAME, "tr")

            header = meclis_sorusturma_imza_sahipleri_tr_tags[0].find_elements(By.TAG_NAME, "th")
            header_items = [item.text for item in header]
            header_items[-1] = "İlk İmza"

            adi_ve_soyadi = [meclis_sorusturma_imza_sahipleri_tr_tags[i].find_elements(By.TAG_NAME, "td")[0].text for i in range(1, len(meclis_sorusturma_imza_sahipleri_tr_tags))]
            partisi = [meclis_sorusturma_imza_sahipleri_tr_tags[i].find_elements(By.TAG_NAME, "td")[1].text for i in range(1, len(meclis_sorusturma_imza_sahipleri_tr_tags))]
            secim_cevresi = [meclis_sorusturma_imza_sahipleri_tr_tags[i].find_elements(By.TAG_NAME, "td")[2].text for i in range(1, len(meclis_sorusturma_imza_sahipleri_tr_tags))]
            ilk_imza = [meclis_sorusturma_imza_sahipleri_tr_tags[i].find_elements(By.TAG_NAME, "td")[3].text for i in range(1, len(meclis_sorusturma_imza_sahipleri_tr_tags))]

            meclis_sorusturma_imza_sahipleri_df = pd.DataFrame([adi_ve_soyadi, partisi, secim_cevresi, ilk_imza]).T.rename({i:header_items[i] for i in range(4)}, axis=1)
            meclis_sorusturma_imza_sahipleri_df.columns = ["adi_ve_soyadi", "partisi", "secim_cevresi", "ilk_imza"]
            meclis_sorusturma_imza_sahipleri_df["meclis_sorusturma_id"] = meclis_sorusturma_id
            meclis_sorusturma_imza_sahipleri_df["ilk_imza"] = meclis_sorusturma_imza_sahipleri_df["ilk_imza"].apply(lambda x: True if x == "İlk İmza Sahibi" else False)

    return meclis_sorusturma_bilgileri_df, meclis_sorusturma_imza_sahipleri_df

In [9]:
if "meclis_sorusturma" not in os.listdir():
    os.mkdir("meclis_sorusturma")

# Create csv files and headers
pd.DataFrame(columns=["donemi_ve_yasama_yili",
                      "esas_numarasi",
                      "baskanliga_gelis_tarihi",
                      "onergenin_ozeti",
                      "okutuldugu_birlesim",
                      "son_durumu",
                      "onergenin_gorusuldugu_tarih",
                      "onergenin_muhatabi",
                      "onerge_sahibi",
                      "meclis_sorusturma_id"]).to_csv(f"meclis_sorusturma/meclis_sorusturma.csv",
                                                   mode="w", index=False)

pd.DataFrame(columns=["adi_ve_soyadi",
                      "partisi",
                      "secim_cevresi",
                      "ilk_imza",
                      "meclis_sorusturma_id"]).to_csv(f"meclis_sorusturma/meclis_sorusturma_imza_sahipleri.csv",
                                                   mode="w", index=False)

In [10]:
# Write to csv files
for i, url in enumerate(urls):
    if i % 5 == 0:
        print(f"{i}/{len(urls)} | {(i)/len(urls)*100:.2f}%")
    meclis_sorusturma, meclis_sorusturma_imza_sahipleri = get_tables(url)
    meclis_sorusturma.to_csv(f"meclis_sorusturma/meclis_sorusturma.csv", mode="a+", index=False, header=False)
    meclis_sorusturma_imza_sahipleri.to_csv(f"meclis_sorusturma/meclis_sorusturma_imza_sahipleri.csv", mode="a+", index=False, header=False)
print(f"{len(urls)}/{len(urls)} | {(len(urls))/len(urls)*100:.2f}%")

0/29 | 0.00%
5/29 | 17.24%
10/29 | 34.48%
15/29 | 51.72%
20/29 | 68.97%
25/29 | 86.21%
29/29 | 100.00%


In [11]:
driver.quit()